In [240]:
import pandas as pd
import numpy as np

In [241]:
data = '/Users/leona/OneDrive/Documentos/GitHub/Machine-learning-studies-2/ensemble/GradientBoosting/exercise/train_titanic.csv'

In [242]:
df = pd.read_csv(data)

In [243]:
df.shape

(891, 12)

In [244]:
df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [245]:
df.isna().sum()/len(df) *100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

In [246]:
df.drop(columns=['Cabin', 'Name', 'Ticket'], axis=1, inplace=True)
df.dropna(subset='Embarked', inplace=True)
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Sex              0
Age            177
SibSp            0
Parch            0
Fare             0
Embarked         0
dtype: int64

In [247]:
for i in df.columns:
    if df[i].dtype == object:
        print(df[i].value_counts(), i)

Sex
male      577
female    312
Name: count, dtype: int64 Sex
Embarked
S    644
C    168
Q     77
Name: count, dtype: int64 Embarked


In [248]:
df['Sex'] = df['Sex'].replace('male', 0)
df['Sex'] = df['Sex'].replace('female', 1)
df = pd.get_dummies(df, columns=['Embarked'], dtype=int)
df.dtypes

C:\Users\leona\AppData\Local\Temp\ipykernel_29576\2856049996.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Sex'] = df['Sex'].replace('female', 1)


PassengerId      int64
Survived         int64
Pclass           int64
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Embarked_C       int32
Embarked_Q       int32
Embarked_S       int32
dtype: object

In [249]:
x_train = df[df['Age'].notna()]
x_train.drop('Age', axis=1, inplace=True)
y_train = df[df['Age'].notna()]['Age']
x_test = df[df['Age'].isna()]
x_test.drop('Age', axis=1, inplace=True)
y_test = df[df['Age'].isna()]['Age']

C:\Users\leona\AppData\Local\Temp\ipykernel_29576\1003052387.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train.drop('Age', axis=1, inplace=True)
C:\Users\leona\AppData\Local\Temp\ipykernel_29576\1003052387.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test.drop('Age', axis=1, inplace=True)


In [250]:
from sklearn.tree import DecisionTreeRegressor

In [251]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

prediction = model.predict(x_test)

In [252]:
df.Age[df['Age'].isna()] = prediction

C:\Users\leona\AppData\Local\Temp\ipykernel_29576\995195806.py:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df.Age[df['Age'].isna()] = prediction
C:\Users\leona\AppData\Local\Temp\ipykernel_29576\995195806.py:1: SettingWithCopyWarning: 


In [253]:
df.isna().sum()/len(df)*100

PassengerId    0.0
Survived       0.0
Pclass         0.0
Sex            0.0
Age            0.0
SibSp          0.0
Parch          0.0
Fare           0.0
Embarked_C     0.0
Embarked_Q     0.0
Embarked_S     0.0
dtype: float64

In [254]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV

In [255]:
x = df.drop(columns='Survived', axis=1)
y = df['Survived']

In [256]:
grid_params = {
    'learning_rate': np.array([0.1, 0.08, 0.2]),
    'min_samples_split': np.array([2, 4]),
    'max_depth': np.array([2, 4, 6]),
    'min_samples_leaf': np.array([2, 4])
}

In [257]:
strat = StratifiedKFold(n_splits=5, shuffle=True, random_state=15)

In [258]:
model = GradientBoostingClassifier(n_estimators=200)
grid = GridSearchCV(estimator=model, param_grid=grid_params, cv=strat, n_jobs=-1)
grid.fit(x, y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=15, shuffle=True),
             estimator=GradientBoostingClassifier(n_estimators=200), n_jobs=-1,
             param_grid={'learning_rate': array([0.1 , 0.08, 0.2 ]),
                         'max_depth': array([2, 4, 6]),
                         'min_samples_leaf': array([2, 4]),
                         'min_samples_split': array([2, 4])})

In [259]:
print(f'The best score of this model was: {grid.best_score_}, with the params: {grid.best_params_}')

The best score of this model was: 0.8324065257411286, with the params: {'learning_rate': 0.1, 'max_depth': 2, 'min_samples_leaf': 4, 'min_samples_split': 2}
